In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load data from Excel file
data_path = 'SPRINGS.xlsx'
data = pd.read_excel(data_path)
data.columns = data.columns.str.strip()

# Define weights and standards for each parameter depenig on the PCA based weights and parameters
parameters = {
    'Aluminum [µg/l Al]': {'weight': 0.001828, 'standard': 200},
    'Ammonium [mg/l NH4]': {'weight': 0.040472, 'standard': 0.5},
    'Arsenic [µg/l As]': {'weight': 0.009174, 'standard': 10},
    'Cadmium [µg/l Cd]': {'weight': 0.033676, 'standard': 5},
    'Chlorides [mg/l Cl]': {'weight': 0.038744, 'standard': 250},
    'Chlorites [µg/l ClO2]': {'weight': 0.018832, 'standard': 700},
    'Copper [mg/l Cu]': {'weight': 0.119593, 'standard': 2},
    'Fluorides [mg/l F]': {'weight': 0.01348, 'standard': 1.5},
    'Hardness [°F]': {'weight': 0.100148, 'standard': 50},
    'Iron [µg/l Fe]': {'weight': 0.040982, 'standard': 200},
    'Lead [µg/l Pb]': {'weight': 0.096853, 'standard': 10},
    'Magnesium [mg/l Mg]': {'weight': 0.111423, 'standard': 30},
    'Manganese [µg/l Mn]': {'weight': 0.04212, 'standard': 50},
    'Nitrates [mg/l NO3]': {'weight': 0.019065, 'standard': 50},
    'pH': {'weight': 0.076599, 'low': 6.5, 'high': 9.5},
    'Sodium [mg/l Na]': {'weight': 0.016428, 'standard': 200},
    'Sulfates [mg/l SO4]': {'weight': 0.078955, 'standard': 250},
    'Turbidity [NTU]': {'weight': 0.049485, 'standard': 0.3},
    'Vanadium [µg/l V]': {'weight': 0.010561, 'standard': 140},
    'Zinc [µg/l Zn]': {'weight': 0.081582, 'standard': 5000}
}

# Calculate sub-indices for each parameter
for param, info in parameters.items():
    if param in data.columns:
        if param == 'pH':
            def ph_index(x):
                # Using 6.5-9.5 range with 7 as optimal value
                return abs((x - 7)/(9.5 - 7)) * 100
            data[f'{param} sub-index'] = data[param].apply(ph_index)
        else:
            # Standard case for other parameters
            data[f'{param} sub-index'] = (data[param] / info['standard']) * 100
            data[f'{param} sub-index'] = data[f'{param} sub-index'].abs()
    else:
        print(f"Warning: {param} not found in dataset")

required_columns = [f'{param} sub-index' for param in parameters if f'{param} sub-index' in data.columns]
data['WQI'] = data[required_columns].mul(
    [parameters[param]['weight'] for param in parameters if f'{param} sub-index' in data.columns]
).sum(axis=1)
# Classify water quality based on WQI
def classify_wqi(wqi):
    if wqi < 50:
        return 'Excellent water'
    elif wqi <= 100:
        return 'Good water'
    elif wqi <= 200:
        return 'Poor water'
    elif wqi <= 300:
        return 'Very poor water'
    else:
        return 'Unsuitable for drinking'
# Apply classification
data['Water Quality Classification'] = data['WQI'].apply(classify_wqi)
classification_counts = data['Water Quality Classification'].value_counts()
classification_percentages = (classification_counts / len(data) * 100).round(2)

classification_summary = pd.DataFrame({
    'Count': classification_counts,
    'Percentage (%)': classification_percentages
})
# Plot histogram of WQI
plt.figure(figsize=(10, 6))
plt.hist(data['WQI'], bins=50, color='blue', alpha=0.7)
plt.title('Distribution of Water Quality Index (WQI)')
plt.xlabel('Water Quality Index')
plt.ylabel('Frequency')
plt.grid(True)
plt.savefig('WQI_Distribution_New.png')
plt.show()
# Save results to a new Excel file
output_path = 'WQI_SPRINGS.xlsx'
with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
    data.to_excel(writer, sheet_name='Detailed Results', index=False)
    classification_summary.to_excel(writer, sheet_name='Classification Summary')
#  Print confirmation
print(f'Results saved to {output_path}')
print("\nWater Quality Classification Summary:")
print(classification_summary)